In [ ]:
import os, torch

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
os.environ['NVIDIA_VISIBLE_DEVICES'] = "1"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

In [ ]:
import sys
import os
from pathlib import Path

import torch
import onnx
import numpy as np
import argparse
import onnx_graphsurgeon as gs

from utils import experiment_loader, initial_logger, copyStateDict, get_cfg_defaults

sys.path.append("../")
from craft import CRAFT

cfg_detec = get_cfg_defaults()
cfg_detec
# weight_path = '../weights'
# model_path, model_config = experiment_loader(model_format='pth', data_path=weight_path)

In [ ]:
class Args():
    def __init__(self,volume_path='../data/Synapse/test_vol_h5',dataset='Synapse',num_classes=9,img_size=224,vit_name='R50-ViT-B_16',n_skip=3,vit_patches_size=16):
        self.volume_path = volume_path
        self.dataset = dataset
        self.num_classes = num_classes
        self.img_size = img_size
        self.vit_name = vit_name
        self.n_skip = n_skip
        self.vit_patches_size = vit_patches_size

args = Args()

In [ ]:
# -*- coding: utf-8 -*-
"""
_____________________________________________________________________________

This file contain code for converting trained model into ONNX format
Refer from TensorRT example: tensorrt/bin/python/onnx_packnet
_____________________________________________________________________________
"""
# from icecream import ic
import sys
import os
from pathlib import Path

import torch
import onnx
import numpy as np
import argparse
import onnx_graphsurgeon as gs

from utils import experiment_loader, initial_logger, copyStateDict, get_cfg_defaults

sys.path.append("../")
from craft import CRAFT

logger = initial_logger()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def build_onnx(weight_path):
    """Load the network and export it to ONNX
    """
    logger.info("Converting detec pth to onnx...")

    model_path, model_config = experiment_loader(model_format='pth', data_path=weight_path)
    
    # Load config come with trained model
    cfg_detec = get_cfg_defaults()
    cfg_detec.merge_from_file(model_config)
    
    # Set output path for onnx files
    output_path = Path('../weights')
    
    # Set name for onnx files
    output_detec = os.path.join(output_path, "detect.onnx")
    
    # Dummy input data for models
    input_tensor_detec = torch.randn((1, 3, 500, 500), requires_grad=False)
    input_tensor_detec=input_tensor_detec.cuda()
    input_tensor_detec=input_tensor_detec.to(device=device)
    
    # Load net
    net = CRAFT()
    net.load_state_dict(copyStateDict(torch.load(model_path)))
    net = net.to(device)
    net.eval()
    
    # Convert the model into ONNX
    torch.onnx.export(net, input_tensor_detec, output_detec,
                      verbose=True, opset_version=cfg_detec.INFERENCE.OX_OPSET,
                      do_constant_folding=cfg_detec.INFERENCE.OX_DO_CONSTANT_FOLDING,
                      export_params=cfg_detec.INFERENCE.OX_EXPORT_PARAMS,
                      input_names=["input"],
                      output_names=["output"],
                      dynamic_axes={"input": {2: "height", 3: "width"}})
#     torch.onnx.export(net, input_tensor_detec, output_detec,
#                       verbose=True, opset_version=cfg_detec.INFERENCE.OX_OPSET,
#                       do_constant_folding=cfg_detec.INFERENCE.OX_DO_CONSTANT_FOLDING,
#                       export_params=cfg_detec.INFERENCE.OX_EXPORT_PARAMS,
#                       input_names=["input"],
#                       output_names=["output", "output1"],
#                       dynamic_axes={"input": {0: "batch", 2: "height", 3: "width"}})

    logger.info("Convert detec pth to ONNX sucess")

In [ ]:

# parser = argparse.ArgumentParser(description="Exports model to ONNX, and post-processes it to insert TensorRT plugins")
# parser.add_argument("--weight", required=False, help="Path to input model folder", default='/workspace/DBP/NAS저장공간/hengbee/ONNX-TensorRT-Inference-CRAFT-pytorch/weights')
# args=parser.parse_args()
# print(args.weight)

weight_path = '../weights'


In [ ]:
build_onnx(weight_path)

In [ ]:
import onnx
import time

onnx_load_befo_mem = torch.cuda.memory_allocated()/1024/1024
print("current_memory:", onnx_load_befo_mem)

t2 = time.time()

output_path = Path('../weights')
# Set name for onnx files
output_detec = os.path.join(output_path, "transunet.onnx")
# Load the ONNX model
model = onnx.load(output_detec)

# Check that the model is well formed
onnx.checker.check_model(model)

print(f'load time: {time.time()-t2}')

onnx_load_mem = torch.cuda.memory_allocated()/1024/1024

print("onnx_load_mem: %fMB"%(onnx_load_mem-onnx_load_befo_mem))

In [ ]:

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

In [ ]:
import onnxruntime as ort
import imgproc
import cv2
from torch.autograd import Variable

onnx_infer_befo_mem = torch.cuda.memory_allocated()/1024/1024
print("current_memory:", onnx_infer_befo_mem)

t = time.time()
image_path = '../IMG_8178.jpg'
print("Test image :", image_path)

# image = imgproc.loadImage(image_path)
ort_session = ort.InferenceSession(output_detec)
# img_resized = imgproc.normalizeMeanVariance(image)
# img_resized = torch.from_numpy(img_resized).permute(2, 0, 1)    # [h, w, c] to [c, h, w]
# img_resized = Variable(img_resized.unsqueeze(0))
# img_resized.cuda()
# print(img_resized.shape)
img = cv2.imread('../images.jpeg',cv2.IMREAD_COLOR)
dsize=(224,224)
img_resized = cv2.resize(img, dsize)
img_resized = torch.from_numpy(img_resized).permute(2, 0, 1)    # [h, w, c] to [c, h, w]
img_resized = Variable(img_resized.unsqueeze(0))
img_resized.cuda()
# print(img_resized.shape)
outputs = ort_session.run(
    None,
    {"input": np.array(img_resized).astype(np.float32)},
)

print(f'load time: {time.time()-t}')
onnx_infer_mem = torch.cuda.memory_allocated()/1024/1024

print("onnx_load_mem: %fMB"%(onnx_infer_mem-onnx_infer_befo_mem))

In [ ]:
import matplotlib.pylab as pl
from matplotlib.colors import ListedColormap

cmap=pl.cm.tab10_r

my_cmap = cmap(np.arange(cmap.N))
my_cmap[:, -1] = 0.9
my_cmap[0, -1] = 0.1
my_cmap = ListedColormap(my_cmap)
print(my_cmap.colors)

In [ ]:
y_class = np.argmax(outputs[0], axis=-1)
y_class

In [ ]:
y_class.shape

In [ ]:
score_seg.shape

In [ ]:
for sc in score_seg:
    alpha=0.8
    plt.imshow(sc, cmap=my_cmap, alpha=alpha)
    plt.show()

In [ ]:
score_text = outputs[0][0,:,:,0]
score_link = outputs[0][0,:,:,1]
score_text

In [ ]:
score_text.shape

In [ ]:
import matplotlib.pyplot as plt
def img_show(img, size =(15,15)):
    plt.rcParams["figure.figsize"] = size
    imgplot = plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
img_show(score_text)